In [23]:
import pymongo
import base64
import numpy as np
from PIL import Image
from io import BytesIO

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["fingerprintDB"]
collection = db["fingerprints"]

# Fetch data
data = collection.find()

# Print the number of records in the collection
print(f"Number of records in the collection: {data.count()}")

# Decode canvas images and convert them to numpy arrays
def decode_canvas(canvas_data):
    image_data = base64.b64decode(canvas_data.split(",")[1])
    image = Image.open(BytesIO(image_data))
    return np.array(image)

# Set the target size for all images (e.g., 35x280)
target_size = (35, 280)

# Function to resize images
def resize_image(image, target_size):
    return np.array(Image.fromarray(image).resize(target_size, Image.LANCZOS))

# Decode all canvas images and resize them to the target size
canvases = []
labels = []
for record in data:
    for canvas in record["canvases"]:
        decoded_image = decode_canvas(canvas)
        resized_image = resize_image(decoded_image, target_size)
        canvases.append(resized_image)
        labels.append(record["username"])

    # Print information about each record
    print(f"Username: {record['username']}, Number of canvases: {len(record['canvases'])}")

canvases = np.array(canvases)
labels = np.array(labels)

# Print the shape of the canvases and labels arrays
print(f"Canvases shape: {canvases.shape}, Labels shape: {labels.shape}")

Number of records in the collection: 2
Username: username_3, Number of canvases: 20
Username: username_4, Number of canvases: 20
Canvases shape: (40, 280, 35, 4), Labels shape: (40,)


c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


In [ ]:
import pymongo
import base64
import numpy as np
from PIL import Image
from io import BytesIO

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["fingerprintDB"]
collection = db["fingerprints"]

# Fetch data
data = collection.find()

# Print the number of records in the collection
print(f"Number of records in the collection: {data.count()}")

# Decode canvas images and convert them to numpy arrays
def decode_canvas(canvas_data):
    image_data = base64.b64decode(canvas_data.split(",")[1])
    image = Image.open(BytesIO(image_data))
    return np.array(image)

# Set the target size for all images (e.g., 35x280)
target_size = (35, 280)

# Function to resize images
def resize_image(image, target_size):
    return np.array(Image.fromarray(image).resize(target_size, Image.LANCZOS))

# Decode all canvas images and resize them to the target size
canvases = []
labels = []
for record in data:
    for canvas in record["canvases"]:
        decoded_image = decode_canvas(canvas)
        resized_image = resize_image(decoded_image, target_size)
        canvases.append(resized_image)
        labels.append(record["username"])

    # Print information about each record
    print(f"Username: {record['username']}, Number of canvases: {len(record['canvases'])}")

canvases = np.array(canvases)
labels = np.array(labels)

# Print the shape of the canvases and labels arrays
print(f"Canvases shape: {canvases.shape}, Labels shape: {labels.shape}")

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Bilder skalieren
canvases = canvases / 255.0

# Labels in numerische Werte umwandeln
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Daten in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(canvases, labels, test_size=0.2, random_state=42)


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Modell erstellen
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(35, 280, 4)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Modell kompilieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Modell zusammenfassen
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 33, 278, 32)       1184      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 16, 139, 32)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 137, 64)       18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 7, 68, 64)        0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 30464)             0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [24]:
print("Form von X_train:", X_train.shape)
print("Form von X_test:", X_test.shape)
canvases = np.transpose(canvases, (0, 2, 1, 3))

Form von X_train: (32, 280, 35, 4)
Form von X_test: (8, 280, 35, 4)


In [25]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(labels)

In [26]:
# Modell trainieren
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10


ValueError: in user code:

    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Lion Witte\anaconda3\envs\newenv\lib\site-packages\keras\engine\input